<a href="https://colab.research.google.com/github/abhijit-exe/abhijit-exe/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A text summarization project**

In [ ]:
!pip install transformers datasets nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split='train[:15%]')  # Load 15% for testing

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Sample Article

In [ ]:
print("Article:")
print(dataset[0]['article'])
print("Summary:")
print(dataset[0]['highlights'])  # This is the summary provided

Article:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Detai

Tokenizer Function

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

3 types: Brief, Medium and Detailed Summary

In [ ]:
def summarize_text(text, length="medium"):
    # Set max_length based on the desired summary length
    if length == "brief":
        max_len = 60  # for a brief summary
        min_len = 30  # minimum length for a brief summary
    elif length == "medium":
        max_len = 120  # for a medium-length summary
        min_len = 80   # minimum length for a medium summary
    else:  # default to "detailed"
        max_len = 180  # for a more detailed summary
        min_len = 120  # minimum length for a detailed summary

    # Tokenize the input text (convert text to model-friendly format)
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary using the model
    summary_ids = model.generate(
        inputs,  # Input tokens
        max_length=max_len,  # Maximum length of the summary
        min_length=min_len,  # Minimum length of the summary (to avoid too short summaries)
        length_penalty=2.0,  # Penalizes the generation of excessive tokens
        num_beams=4,  # Number of beams for beam search (increases quality)
        # early_stopping=True  # Stop early if the summary reaches an optimal length
    )

    # Decode the generated token IDs back into human-readable text
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
sample_text = dataset[1]['article']

# Test the summarizer with different lengths
brief_summary = summarize_text(sample_text, length="brief")
medium_summary = summarize_text(sample_text, length="medium")
detailed_summary = summarize_text(sample_text, length="detailed")

print("Brief Summary:", brief_summary)
print("Medium Summary:", medium_summary)
print("Detailed Summary:", detailed_summary)

Brief Summary: inmates with most severe mental illnesses are incarcerated until they're ready to appear in court. judge says arrests often result from confrontations with police. prisoners are wearing sleeveless robes, wearing sleeveless robes
Medium Summary: inmates with most severe mental illnesses are incarcerated until they're ready to appear in court. judge says arrests often result from confrontations with police. prisoners are wearing sleeveless robes, but not getting any real help because they're in jail. "forgotten floor" is dubbed the "forgotten floor" by soledad o'brien.
Detailed Summary: inmates with most severe mental illnesses are incarcerated until they're ready to appear in court. judge says arrests often result from confrontations with police. prisoners are wearing sleeveless robes, but not getting any real help because they're in jail. "forgotten floor" is dubbed the "forgotten floor," where many mentally ill inmates are housed a day before trial - judge says a third 